In [ ]:
import pandas as pd
data = pd.read_csv("stage3.csv")

datum = data["date"]

for i in range(len(data)):
    if "2013" in datum[i]:
        data = data.drop([i])

data = data.drop(["incident_url", "source_url", "congressional_district", "incident_url_fields_missing", "n_guns_involved", "sources", "state_house_district", "state_senate_district"], axis=1)

data.to_csv("data_clean.csv")



In [2]:
import pandas as pd
clean = pd.read_csv("data_clean.csv")
date = clean["date"]
n_killed = clean["n_killed"]
n_injured = clean["n_injured"]
city = clean["city_or_county"]
state = clean["state"]
killcity = {}
killstate = {}
injuredcity = {}
injuredstate = {}


killed2014 = 0
killed2015 = 0
killed2016 = 0
killed2017 = 0
killed2018 = 0 
injured2014 = 0
injured2015 = 0
injured2016 = 0
injured2017 = 0
injured2018 = 0

for i in range(len(clean)):
#     if city[i] not in killcity:
#         killcity[city[i]] = 0
#     if city[i] in killcity:
#         killcity[city[i]] += n_killed[i]
#     if state[i] not in killstate:
#         killstate[state[i]] = 0
#     if state[i] in killstate:
#         killstate[state[i]] += n_killed[i]
#     if city[i] not in injuredcity:
#         injuredcity[city[i]] = 0
#     if city[i] in injuredcity:
#         injuredcity[city[i]] += n_injured[i]
#     if state[i] not in injuredstate:
#         injuredstate[state[i]] = 0
#     if state[i] in injuredstate:
#         injuredstate[state[i]] += n_injured[i]
    if "2014" in date[i]:
        injured2014 += n_injured[i]
    if "2015" in date[i]:
        injured2015 += n_injured[i]
    if "2016" in date[i]:
        injured2016 += n_injured[i]    
    if "2017" in date[i]:
        injured2017 += n_injured[i]
    if "2018" in date[i]:
        injured2018 += n_injured[i]  
    if "2014" in date[i]:
        killed2014 += n_killed[i]
    if "2015" in date[i]:
        killed2015 += n_killed[i]
    if "2016" in date[i]:
        killed2016 += n_killed[i]    
    if "2017" in date[i]:
        killed2017 += n_killed[i]
    if "2018" in date[i]:
        killed2018 += n_killed[i]    
        


# [2013_killed+= n_killed[i[]] for i in range(len(data)) if "2013" in date[i]  ]



In [ ]:
# print(killed2014, killed2015, killed2016, killed2017, (killed2018/90)*365)
# print(injured2014/365, injured2015/365, injured2016/366, injured2017/365, injured2018/90)
# print(killcity)
# print(killstate)
# print(injuredcity)
# print(injuredstate)
print(len(list(killcity.keys())))
print("murder per day 2014-2018: 34.402739726 36.9424657534 41.1639344262 42.495890411 39.2555555556")
print("injuries per day 2014-2018: 63.0191780822 73.8821917808 83.5519125683 84.1178082192 68.5666666667")
print("murders per year 2014-2018: 2014 = 12557, 2015 = 13484, 2016 = 15066, 2017 = 15511, 2018 = 3533")
print("injuries per year 2014-2018: 2014 = 23002, 2015 = 26967, 2016 = 30580, 2017 = 30703, 2018 = 6171")
print("highest number of murders was in the city/county Chicago (2098)")
print("highest number of murders was in the state of California (5516)")
print("highest number of injuries was in the city/county Chicago (10356)")
print("highest number of injuries was in the state Illinois (13436)")

In [87]:
from pygeocoder import Geocoder
import pandas as pd
import numpy as np
# print(list(killcity.keys()))
longlatkillcity = []
longlatkillcityrejects =[]
for i in list(killcity.keys()):
    try:
        print(Geocoder.geocode(str(i)).coordinates)
        longlatkillcity.append(Geocoder.geocode(str(i)).coordinates)
    except:
        longlatkillcityrejects.append(i)
    
# results.coordinates

# adres = data["address"]
# long  = data["longitude"]
# lat = data["latitude"]
# count = 0
# for i in range(len(data)):
#     if pd.isna(adres[i]):
#         count+=1
        
# print(count)

# results = Geocoder.reverse_geocode(lat[2], long[2])
# print(results.street_address)

In [12]:
# print(longlatkillcity)
# print(longlatkillcityrejects)
murderyears = []
injuredyears = []
murderyears.extend((killed2014, killed2015, killed2016, killed2017, killed2018))
injuredyears.extend((injured2014, injured2015, injured2016, injured2017, injured2018))

In [13]:
from bokeh.transform import factor_cmap
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.models import FactorRange
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool

output_file("murders_injuries_per_year.html")

years = ['2014', '2015', '2016', '2017', '2018']

data = {'years' : [2014,2015,2016,2017,2018], 
        'murders' : murderyears,
        'injuries' : injuredyears}


x = [("2014", "Murders"), ("2014", "Injuries"), ("2015", "Murders"), ("2015", "Injuries"), ("2016", "Murders"), ("2016", "Injuries"), ("2017", "Murders"), ("2017", "Injuries"), ("2018", "Murders"), ("2018","Injuries")]
counts = sum(zip(data['murders'], data['injuries']), ()) # like an hstack

murinj =[]

for g in range(5):
    murinj.append(murderyears[g])
    murinj.append(injuredyears[g])


source = ColumnDataSource(data=dict(
    x=x, 
    counts=counts,
    value = murinj,
))

hover = HoverTool(tooltips=[
    ("value", "@value"),
])

p = figure(x_range=FactorRange(*x), plot_height=250, title="Murders/Injuries per Year", tools=[hover])

p.vbar(x='x', top='counts', width=0.9, source=source,
       fill_color=factor_cmap('x', palette=["crimson", "darkorange"], factors=["Murders", "Injuries"], start=1, end=2))

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)